<a href="https://colab.research.google.com/github/AhabbscienceStudioPak/siggraphasia2019_remastering/blob/master/Copy_of_remastering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Dec 24 19:15:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/AhabbscienceStudioPak/siggraphasia2019_remastering

Cloning into 'siggraphasia2019_remastering'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), done.


In [ ]:
%cd siggraphasia2019_remastering

!bash download_model.sh

/content/siggraphasia2019_remastering
--2020-12-24 19:26:52--  http://iizuka.cs.tsukuba.ac.jp/data/remasternet.pth.tar
Resolving iizuka.cs.tsukuba.ac.jp (iizuka.cs.tsukuba.ac.jp)... 130.158.81.20
Connecting to iizuka.cs.tsukuba.ac.jp (iizuka.cs.tsukuba.ac.jp)|130.158.81.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256796033 (245M) [application/x-tar]
Saving to: ‘model/remasternet.pth.tar’

model/remasternet.p 100%[===================>] 244.90M  18.7MB/s    in 14s     

2020-12-24 19:27:07 (17.3 MB/s) - ‘model/remasternet.pth.tar’ saved [256796033/256796033]

Checking integrity (md5sum)...
Download finished successfully!


Create directory for video input and output + for coloured references

In [ ]:
!mkdir -p /content/siggraphasia2019_remastering/video_input
!mkdir -p /content/siggraphasia2019_remastering/result
!mkdir -p /content/siggraphasia2019_remastering/reference_images

Add files from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Add video or reference images from YouTube

In [ ]:
!pip install youtube_dl

     |████████████████████████████████| 1.8MB 6.9MB/s 


In [ ]:
%cd /content/siggraphasia2019_remastering/video_input
!youtube-dl -f 18 -o "/content/siggraphasia2019_remastering/video_input/input.mp4" https://www.youtube.com/watch?v=ljwTaMfORzs 

Upload your video directly to google colab

In [ ]:
%cd /content/siggraphasia2019_remastering/video_input

from google.colab import files
uploaded = files.upload() 
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

/content/siggraphasia2019_remastering/video_input


Saving quaidcrop.mp4 to quaidcrop.mp4
User uploaded file "quaidcrop.mp4" with 11685750 bytes


Upload your colored references directly to Colab



In [ ]:
%cd /content/siggraphasia2019_remastering/reference_images

from google.colab import files
uploaded = files.upload() 
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

/content/siggraphasia2019_remastering/reference_images


Saving quaid00001.png to quaid00001.png
Saving quaid00002.png to quaid00002.png
Saving quaid00003.png to quaid00003.png
Saving quaid00004.png to quaid00004.png
Saving quaid00121.png to quaid00121.png
Saving quaid00122.png to quaid00122.png
Saving quaid00123.png to quaid00123.png
Saving quaid00124.png to quaid00124.png
User uploaded file "quaid00001.png" with 2342812 bytes
User uploaded file "quaid00002.png" with 2455483 bytes
User uploaded file "quaid00003.png" with 2427720 bytes
User uploaded file "quaid00004.png" with 2464026 bytes
User uploaded file "quaid00121.png" with 2382882 bytes
User uploaded file "quaid00122.png" with 2340532 bytes
User uploaded file "quaid00123.png" with 2381000 bytes
User uploaded file "quaid00124.png" with 2364798 bytes


If you have to change video aspect to match with the reference file (optional)



In [ ]:
%cd /content/siggraphasia2019_remastering/video_input
!pip install moviepy
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
ffmpeg_extract_subclip("input.mp4", 12, 67, targetname="input_crop.mp4")


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i quaidvideo.mp4 -ss 12.00 -t 55.00 -vcodec copy -acodec copy quaidcrop.mp4
... command successful.


In [ ]:
%cd /content/siggraphasia2019_remastering

/content/siggraphasia2019_remastering


In [ ]:

%%writefile remaster.py
"""
   Copyright (C) <2019> <Satoshi Iizuka and Edgar Simo-Serra>
   This work is licensed under the Creative Commons
   Attribution-NonCommercial-ShareAlike 4.0 International License. To view a copy
   of this license, visit http://creativecommons.org/licenses/by-nc-sa/4.0/ or
   send a letter to Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.
   Satoshi Iizuka, University of Tsukuba
   iizuka@aoni.waseda.jp, http://iizuka.cs.tsukuba.ac.jp/index_eng.html
   Edgar Simo-Serra, Waseda University
   ess@waseda.jp, https://esslab.jp/~ess/
"""

import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm
import os
import argparse
import subprocess
import utils

parser = argparse.ArgumentParser(description='Remastering')
parser.add_argument('--input',   type=str,   default='none', help='Input video')
parser.add_argument('--reference_dir',  type=str, default='none', help='Path to the reference image directory')
parser.add_argument('--disable_colorization', action='store_true', default=False, help='Remaster without colorization')
parser.add_argument('--gpu',       action='store_true', default=False, help='Use GPU')
parser.add_argument('--mindim',     type=int,   default='320',    help='Length of minimum image edges')
opt = parser.parse_args()

device = torch.device('cuda:0' if opt.gpu else 'cpu')

# Load remaster network
modelR = __import__( 'model.remasternet', fromlist=['NetworkR'] ).NetworkR()
state_dict = torch.load( 'model/remasternet.pth.tar' )
modelR.load_state_dict( state_dict['modelR'] )
modelR = modelR.to(device)
modelR.eval()
if not opt.disable_colorization:
   modelC = __import__( 'model.remasternet', fromlist=['NetworkC'] ).NetworkC()
   modelC.load_state_dict( state_dict['modelC'] )
   modelC = modelC.to(device)
   modelC.eval()

print('Processing %s...'%os.path.basename(opt.input))

outputdir = 'tmp/'
outputdir_in = outputdir+'input/'
os.makedirs( outputdir_in, exist_ok=True )
outputdir_out = outputdir+'siggraphasia2019_remastering/'
os.makedirs( outputdir_out, exist_ok=True )

# Prepare reference images
if not opt.disable_colorization:
   if opt.reference_dir!='none':
      import glob
      ext_list = ['png','jpg','bmp']
      reference_files = []
      for ext in ext_list:
         reference_files += glob.glob( opt.reference_dir+'/*.'+ext, recursive=True )
      aspect_mean = 0
      minedge_dim = 256
      refs = []
      for v in reference_files:
         refimg = Image.open( v ).convert('RGB')
         w, h = refimg.size
         aspect_mean += w/h
         refs.append( refimg )
      aspect_mean /= len(reference_files)
      target_w = int(256*aspect_mean) if aspect_mean>1 else 256
      target_h = 256 if aspect_mean>=1 else int(256/aspect_mean)
      refimgs = torch.FloatTensor(len(reference_files), 3, target_h, target_w)
      for i, v in enumerate(refs):
         refimg = utils.addMergin( v, target_w=target_w, target_h=target_h )
         refimgs[i] = transforms.ToTensor()( refimg )
      refimgs = refimgs.view(1, refimgs.size(0), refimgs.size(1), refimgs.size(2), refimgs.size(3)).to( device )

# Load video
cap = cv2.VideoCapture( opt.input )
nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # cv2.CAP_PROP_FRAME_COUNT: 7
v_w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
v_h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
minwh = min(v_w,v_h)
scale = 1
if minwh != opt.mindim:
   scale = opt.mindim / minwh
t_w = round(v_w*scale/16.)*16
t_h = round(v_h*scale/16.)*16
fps = cap.get(cv2.CAP_PROP_FPS)
pbar = tqdm(total=nframes)
block = 5

# Process 
with torch.no_grad():
   it = 0
   while True:
      frame_pos = it*block
      if frame_pos >= nframes:
         break
      cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
      if block >= nframes-frame_pos:
         proc_g = nframes-frame_pos
      else:
         proc_g = block

      input = None
      gtC = None
      for i in range(proc_g):
         index = frame_pos + i
         _, frame = cap.read()
         frame = cv2.resize(frame, (t_w, t_h))
         nchannels = frame.shape[2]
         if nchannels == 1 or not opt.disable_colorization:
            frame_l = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            cv2.imwrite(outputdir_in+'%07d.png'%index, frame_l)
            frame_l = torch.from_numpy(frame_l).view( frame_l.shape[0], frame_l.shape[1], 1 )
            frame_l = frame_l.permute(2, 0, 1).float() # HWC to CHW
            frame_l /= 255.
            frame_l = frame_l.view(1, frame_l.size(0), 1, frame_l.size(1), frame_l.size(2))
         elif nchannels == 3:
            cv2.imwrite(outputdir_in+'%07d.png'%index, frame)
            frame = frame[:,:,::-1] ## BGR -> RGB
            frame_l, frame_ab = utils.convertRGB2LABTensor( frame )
            frame_l = frame_l.view(1, frame_l.size(0), 1, frame_l.size(1), frame_l.size(2))
            frame_ab = frame_ab.view(1, frame_ab.size(0), 1, frame_ab.size(1), frame_ab.size(2))

         input = frame_l if i==0 else torch.cat( (input, frame_l), 2 )
         if nchannels==3 and opt.disable_colorization:
            gtC = frame_ab if i==0 else torch.cat( (gtC, frame_ab), 2 )
      
      input = input.to( device )

      # Perform restoration
      output_l = modelR( input ) # [B, C, T, H, W]

      # Save restoration output without colorization when using the option [--disable_colorization]
      if opt.disable_colorization:
         for i in range( proc_g ):
            index = frame_pos + i
            if nchannels==3:
               out_l = output_l.detach()[0,:,i].cpu()
               out_ab = gtC[0,:,i].cpu()
               out = torch.cat((out_l, out_ab),dim=0).detach().numpy().transpose((1, 2, 0))
               out = Image.fromarray( np.uint8( utils.convertLAB2RGB( out )*255 ) )
               out.save( outputdir_out+'%07d.png'%(index) )
            else:
               save_image( output_l.detach()[0,:,i], outputdir_out+'%07d.png'%(index), nrow=1 )
      # Perform colorization
      else:
         if opt.reference_dir=='none':
            output_ab = modelC( output_l )
         else:
            output_ab = modelC( output_l, refimgs )
         output_l = output_l.detach().cpu()
         output_ab = output_ab.detach().cpu()
         
         # Save output frames of restoration with colorization
         for i in range( proc_g ):
            index = frame_pos + i
            out_l = output_l[0,:,i,:,:]
            out_c = output_ab[0,:,i,:,:]
            output = torch.cat((out_l, out_c), dim=0).numpy().transpose((1, 2, 0))
            output = Image.fromarray( np.uint8( utils.convertLAB2RGB( output )*255 ) )
            output.save( outputdir_out+'%07d.png'%index )

      it = it + 1
      pbar.update(proc_g)
   
   # Save result videos
   outfile = opt.input.split('/')[-1].split('.')[0]
   cmd = 'ffmpeg -y -r %d -i %s%%07d.png -vcodec libx264 -pix_fmt yuv420p -r %d %s_in.mp4' % (fps, outputdir_in, fps, outfile )
   subprocess.call( cmd, shell=True )
   cmd = 'ffmpeg -y -r %d -i %s%%07d.png -vcodec libx264 -pix_fmt yuv420p -r %d %s_out.mp4' % (fps, outputdir_out, fps, outfile )
   subprocess.call( cmd, shell=True )
   cmd = 'ffmpeg -y -i %s_in.mp4 -vf "[in] pad=2.01*iw:ih [left];movie=%s_out.mp4[right];[left][right] overlay=main_w/2:0,scale=2*iw/2:2*ih/2[out]" %s_comp.mp4' % ( outfile, outfile, outfile )
   subprocess.call( cmd, shell=True )

   import shutil
   shutil.rmtree(outputdir)
   cap.release()
   pbar.close()

Overwriting remaster.py


In [ ]:
%cd /content/siggraphasia2019_remastering
!python remaster.py --input /content/siggraphasia2019_remastering/video_input/quaidcrop.mp4 --reference_dir /content/siggraphasia2019_remastering/reference_images --gpu

/content/siggraphasia2019_remastering
Processing quaidcrop.mp4...
100% 1363/1363 [09:35<00:00,  2.64it/s]ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-li